In [153]:
from datetime import datetime, timedelta, date
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import os

In [154]:
sg = Subgrounds()
holders = sg.load_api('https://api.studio.thegraph.com/query/28103/token-holders/0.0.43')

In [155]:
"""
Fetches the snapshots for the given date range
"""
def do_snapshot_query(holders, start_date: datetime, finish_date: datetime):
    snapshot_query = holders.Query.tokenDailySnapshots(
        orderBy=holders.TokenDailySnapshot.date,
        orderDirection="desc",
        first=1000,
        where={
            "date_gte": start_date.strftime("%Y-%m-%d"),
            "date_lt": finish_date.strftime("%Y-%m-%d"),
            "token_": {
                "name": "gOHM",
            },
        }
    )

    results = sg.query_json([snapshot_query.date, snapshot_query.token.name, snapshot_query.balancesList])
    # [{"randomQueryKey": [{ <record one>}, { <record two> }, ...]}]
    results_query = results[0]
    return results_query.get(list(results_query.keys())[0])

start_date = datetime(2021,12,13)
finish_date = datetime(2021,12,16)
results = do_snapshot_query(holders, start_date, finish_date)

In [165]:
from itertools import islice

"""
Fetches the TokenHolderBalance records specified in the balances parameter
"""
def do_balances_query(holders, balances):
    # Split balances into arrays < 1000 records
    balances_iter = iter(balances)
    balances_chunks = list(iter(lambda: tuple(islice(balances_iter, 1000)), ()))

    df = pd.DataFrame()

    # Grab balances
    for chunk in balances_chunks:
        query = holders.Query.tokenHolderBalances(
            first=1000,
            where={
                "id_in": list(chunk)
            }
        )
#        print("query = ", query)

        results = sg.query_df([query.date, query.holder.holder, query.balance])
#        print("count = ", len(results))
        df = pd.concat([df, results])

    # Combine
    return df

balances_15 = results[0].get("balancesList")
balances_14 = results[1].get("balancesList")
balances_13 = results[2].get("balancesList")

# Format with 6 decimal places
pd.options.display.float_format = "{:.6f}".format

records_15 = do_balances_query(holders, balances_15)
# print(records_15.head())
records_14 = do_balances_query(holders, balances_14)
# print(records_14.head())
records_13 = do_balances_query(holders, balances_13)
# print(records_13.head())

records = pd.concat([records_15, records_14, records_13])
records.head()

  tokenHolderBalances_date           tokenHolderBalances_holder_holder  \
0               2021-12-15  0x00000000008c4fb1c916e0c88fd4cc402d935e7d   
1               2021-12-15  0x0021dd0e73077bf0710dc016ac459ffcc7fc7bae   
2               2021-12-15  0x004a879c8f9dfe208e927f5189fdc0d5ea31da21   
3               2021-12-15  0x00b9022ebc05ba31b48df00477ac385799249f58   
4               2021-12-15  0x0131aafc1e85adae7e2e0cb990cefc7e6848d327   

   tokenHolderBalances_balance  
0                     0.000000  
1                     0.003080  
2                     0.337807  
3                     0.273990  
4                     0.108202  
  tokenHolderBalances_date           tokenHolderBalances_holder_holder  \
0               2021-12-14  0x00000000008c4fb1c916e0c88fd4cc402d935e7d   
1               2021-12-14  0x0021dd0e73077bf0710dc016ac459ffcc7fc7bae   
2               2021-12-14  0x00b9022ebc05ba31b48df00477ac385799249f58   
3               2021-12-14  0x0131aafc1e85adae7e2e0cb990cefc7

,tokenHolderBalances_date,tokenHolderBalances_holder_holder,tokenHolderBalances_balance
0,2021-12-15,0x00000000008c4fb1c916e0c88fd4cc402d935e7d,0.000000
1,2021-12-15,0x0021dd0e73077bf0710dc016ac459ffcc7fc7bae,0.003080
2,2021-12-15,0x004a879c8f9dfe208e927f5189fdc0d5ea31da21,0.337807
3,2021-12-15,0x00b9022ebc05ba31b48df00477ac385799249f58,0.273990
4,2021-12-15,0x0131aafc1e85adae7e2e0cb990cefc7e6848d327,0.108202


In [166]:
import matplotlib.pyplot as plt

filtered_records = records[records["tokenHolderBalances_balance"] > 0.1]

filtered_records.pivot(index="tokenHolderBalances_date", columns="tokenHolderBalances_holder_holder", values="tokenHolderBalances_balance").plot(legend=False)

plt.show()

KeyboardInterrupt: 